In [4]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross, MeanReversionLongOnly
from src.utils import load_data
warnings.filterwarnings('ignore')

In [12]:
ticker = "ETH-USD"
period = "1y"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [13]:
bt_sma = Backtest(data, SmaCross, cash=10_000, commission=0.004, trade_on_close=False)
stats = bt_sma.run()
stats

Start                     2023-04-06 00:00...
End                       2024-04-06 00:00...
Duration                    366 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]               81.159673
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [14]:
bt_sma.plot()

Row(id='2093', ...)

In [15]:
%%time
stats = bt_sma.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

CPU times: user 2.14 s, sys: 14 ms, total: 2.15 s
Wall time: 2.15 s


Start                     2023-04-06 00:00...
End                       2024-04-06 00:00...
Duration                    366 days 00:00:00
Exposure Time [%]                   74.659401
Equity Final [$]                 17819.956304
Equity Peak [$]                  21556.908208
Return [%]                          78.199563
Buy & Hold Return [%]               81.159673
Return (Ann.) [%]                   77.639399
Volatility (Ann.) [%]               72.283053
Sharpe Ratio                         1.074102
Sortino Ratio                        3.255882
Calmar Ratio                         3.683133
Max. Drawdown [%]                  -21.079714
Avg. Drawdown [%]                   -4.919246
Max. Drawdown Duration       36 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                      85.712075
Worst Trade [%]                     -1.566807
Avg. Trade [%]                    

In [16]:
bt_sma.plot()

Row(id='2723', ...)

In [20]:
%%time
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0, trade_on_close=False)
stats = bt_momentum.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

CPU times: user 787 ms, sys: 4.96 ms, total: 792 ms
Wall time: 790 ms


Start                     2023-04-06 00:00...
End                       2024-04-06 00:00...
Duration                    366 days 00:00:00
Exposure Time [%]                   97.820163
Equity Final [$]                 18541.661499
Equity Peak [$]                  20936.762329
Return [%]                          85.416615
Buy & Hold Return [%]               81.159673
Return (Ann.) [%]                   84.793779
Volatility (Ann.) [%]                85.26015
Sharpe Ratio                          0.99453
Sortino Ratio                        3.187883
Calmar Ratio                          3.08072
Max. Drawdown [%]                  -27.524014
Avg. Drawdown [%]                   -5.684425
Max. Drawdown Duration      207 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                  359
Win Rate [%]                        50.417827
Best Trade [%]                      12.288746
Worst Trade [%]                    -11.259537
Avg. Trade [%]                    

In [21]:
bt_momentum.plot()

Row(id='3549', ...)

In [23]:
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0, trade_on_close=True)
stats = bt_momentum.run()
stats

Start                     2023-04-06 00:00...
End                       2024-04-06 00:00...
Duration                    366 days 00:00:00
Exposure Time [%]                   98.637602
Equity Final [$]                 11978.875977
Equity Peak [$]                  12417.980469
Return [%]                           19.78876
Buy & Hold Return [%]               81.159673
Return (Ann.) [%]                   19.670948
Volatility (Ann.) [%]               53.717595
Sharpe Ratio                         0.366192
Sortino Ratio                        0.677488
Calmar Ratio                         0.511346
Max. Drawdown [%]                  -38.468923
Avg. Drawdown [%]                  -11.403995
Max. Drawdown Duration      213 days 00:00:00
Avg. Drawdown Duration       52 days 00:00:00
# Trades                                  362
Win Rate [%]                        49.447514
Best Trade [%]                      10.243561
Worst Trade [%]                    -12.239237
Avg. Trade [%]                    